# Capstone Project

In [2]:
#Import libraries
import pandas as pd
import numpy as np
from urllib.request import urlopen
# Import BeautifulSoup from bs4
from bs4 import BeautifulSoup

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [3]:
#Store the url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

In [4]:
# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, "lxml")
type(soup)

bs4.BeautifulSoup

In [5]:
#Get the text
text = soup.get_text()
#print(text)

From these object extract the table data

In [6]:
# Print the first 10 rows 
rows = soup.find_all('tr')
print(rows[:10])

[<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>, <tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>, <tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>, <tr>
<td>M6A</td>
<td><a href="/wiki/North_York" ti

In [7]:
#Create a loop that iterate through table rows and print out the cells of the rows
for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)

[<td align="center" style="border: 1px solid #FF0000; background-color: #FFE0E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_A" title="List of postal codes of Canada: A">A</a>
</td>, <td align="center" style="border: 1px solid #FF4000; background-color: #FFE8E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_B" title="List of postal codes of Canada: B">B</a>
</td>, <td align="center" style="border: 1px solid #FF8000; background-color: #FFF0E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_C" title="List of postal codes of Canada: C">C</a>
</td>, <td align="center" style="border: 1px solid #FFC000; background-color: #FFF8E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_E" title="List of postal codes of Canada: E">E</a>
</td>, <td align="center" style="border: 1px solid #FFFF00; background-color: #FFFFE0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_code

bs4.element.ResultSet

In [8]:
#Use beautiful soup to to remove the html tags
str_cells = str(row_td)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
print(cleantext)

[A
, B
, C
, E
, G
, H
, J
, K
, L
, M
, N
, P
, R
, S
, T
, V
, X
, Y
]


In [9]:
import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

[A
, B
, C
, E
, G
, H
, J
, K
, L
, M
, N
, P
, R
, S
, T
, V
, X
, Y
]


str

In [10]:
df = pd.DataFrame(data = list_rows)
df.head(10)

,0
0,[]
1,"[M1A, Not assigned, Not assigned\n]"
2,"[M2A, Not assigned, Not assigned\n]"
3,"[M3A, North York, Parkwoods\n]"
4,"[M4A, North York, Victoria Village\n]"
5,"[M5A, Downtown Toronto, Harbourfront\n]"
6,"[M5A, Downtown Toronto, Regent Park\n]"
7,"[M6A, North York, Lawrence Heights\n]"
8,"[M6A, North York, Lawrence Manor\n]"
9,"[M7A, Queen's Park, Not assigned\n]"


In [11]:
df1 = df[0].str.split(',', expand=True)
df1.head(10)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,[],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,[M1A,Not assigned,Not assigned\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,[M2A,Not assigned,Not assigned\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,[M3A,North York,Parkwoods\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,[M4A,North York,Victoria Village\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,[M5A,Downtown Toronto,Harbourfront\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,[M5A,Downtown Toronto,Regent Park\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,[M6A,North York,Lawrence Heights\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,[M6A,North York,Lawrence Manor\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,[M7A,Queen's Park,Not assigned\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [12]:
df1 = df1.ix[1:, :2]

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [13]:
df1[0] = df1[0].str.strip('[')
df1.head(10)

,0,1,2
1,M1A,Not assigned,Not assigned\n]
2,M2A,Not assigned,Not assigned\n]
3,M3A,North York,Parkwoods\n]
4,M4A,North York,Victoria Village\n]
5,M5A,Downtown Toronto,Harbourfront\n]
6,M5A,Downtown Toronto,Regent Park\n]
7,M6A,North York,Lawrence Heights\n]
8,M6A,North York,Lawrence Manor\n]
9,M7A,Queen's Park,Not assigned\n]
10,M8A,Not assigned,Not assigned\n]


In [14]:
df1[2] = df1[2].str.strip('\n]')
df1.head(10)

,0,1,2
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


In [16]:
df1.columns = ["Postcode", "Borough", "Neighborhood"]


In [17]:
df1.head()

,Postcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [18]:
df=df1[df1['Borough']!='Not assigned']

In [19]:
df1.loc[df1['Neighborhood']=='Not assigned', ['Neighborhood']]=df1['Borough']

In [20]:
geodata = pd.read_csv("https://cocl.us/Geospatial_data")
geodata.rename(columns = {'Postal Code' : 'Postcode'}, inplace = True)
geodata.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
#Merge dataframes
df= pd.merge(df1,geodata,
                 on='Postcode', 
                 how='left')
df = df.dropna()

In [22]:
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
6,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [23]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 212 neighborhoods.


In [24]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [25]:
# create map of Toronto using latitude and longitude values
map_tr = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tr)  
    
map_tr

In [29]:

wt_data = df[df['Borough'] == ' Downtown Toronto'].reset_index(drop=True)
wt_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


## Explore on Downtown Torornto

I am here working with Downtown Torornto only. Lets explore!

In [30]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="wt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [31]:
# create map of Downtown Toronto using latitude and longitude values
map_wt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(wt_data['Latitude'], wt_data['Longitude'], wt_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_wt)  
    
map_wt

In [32]:
# @hidden_cell
CLIENT_ID = 'PDV131Y2CVTPXUR5G1U54PCUL20DUOFQWNXA5SACNP2ROQU5' # your Foursquare ID
CLIENT_SECRET = 'RVQEBO0K05YMWX1OHKJQXO05G1JLCTAQUJA1ZYYSRUPWIGRW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [33]:
wt_data.loc[0, 'Neighborhood']

' Harbourfront'

In [34]:
neighborhood_latitude = wt_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = wt_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = wt_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of  Harbourfront are 43.6542599, -79.3606359.


In [35]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=PDV131Y2CVTPXUR5G1U54PCUL20DUOFQWNXA5SACNP2ROQU5&client_secret=RVQEBO0K05YMWX1OHKJQXO05G1JLCTAQUJA1ZYYSRUPWIGRW&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [36]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c6332aa6a60712d346ecd28'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16a941735',
         'name': 'Bakery',
         'pluralName': 'Bakeries',
         'primary': True,
         'shortName': 'Bakery'}],
       'id': '54ea41ad498e9a11e9e13308',
       'location': {'address': '362 King St E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Trinity St',
        'distance': 143,
        'formattedAddress': ['362 King St E (Trinity St)',
         'Toronto ON M5A 1K9',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

In [37]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [39]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

47 venues were returned by Foursquare.


In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
wt_venues = getNearbyVenues(names=wt_data['Neighborhood'],
                                   latitudes=wt_data['Latitude'],
                                   longitudes=wt_data['Longitude']
                                  )


 Harbourfront
 Regent Park
 Ryerson
 Garden District
 St. James Town
 Berczy Park
 Central Bay Street
 Christie
 Adelaide
 King
 Richmond
 Harbourfront East
 Toronto Islands
 Union Station
 Design Exchange
 Toronto Dominion Centre
 Commerce Court
 Victoria Hotel
 Harbord
 University of Toronto
 Chinatown
 Grange Park
 Kensington Market
 CN Tower
 Bathurst Quay
 Island airport
 Harbourfront West
 King and Spadina
 Railway Lands
 South Niagara
 Rosedale
 Stn A PO Boxes 25 The Esplanade
 Cabbagetown
 St. James Town
 First Canadian Place
 Underground city
 Church and Wellesley


In [42]:
print(wt_venues.shape)
wt_venues.head()

(2488, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [43]:
wt_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,14,14,14,14,14,14
Berczy Park,54,54,54,54,54,54
CN Tower,14,14,14,14,14,14
Cabbagetown,44,44,44,44,44,44
Central Bay Street,83,83,83,83,83,83
Chinatown,99,99,99,99,99,99
Christie,16,16,16,16,16,16
Church and Wellesley,89,89,89,89,89,89


In [44]:
print('There are {} uniques categories.'.format(len(wt_venues['Venue Category'].unique())))

There are 207 uniques categories.


In [45]:
# one hot encoding
wt_onehot = pd.get_dummies(wt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
wt_onehot['Neighborhood'] = wt_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [wt_onehot.columns[-1]] + list(wt_onehot.columns[:-1])
wt_onehot = wt_onehot[fixed_columns]

wt_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
wt_onehot.shape

(2488, 207)

In [47]:
wt_grouped = wt_onehot.groupby('Neighborhood').mean().reset_index()
wt_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.010000
1,Bathurst Quay,0.000000,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,CN Tower,0.000000,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Central Bay Street,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.012048,0.000000,0.000000,0.012048,0.000000,0.000000
6,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010101,0.00,0.00,0.00,0.050505,0.000000,0.050505,0.010101,0.000000,0.000000
7,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Church and Wellesley,0.011236,0.011236,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.011236,0.011236,0.000000,0.011236,0.000000
9,Commerce Court,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000


In [48]:
wt_grouped.shape

(36, 207)

In [49]:
num_top_venues = 5

for hood in wt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = wt_grouped[wt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3           Steakhouse  0.04
4      Thai Restaurant  0.04


---- Bathurst Quay----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3             Plane  0.07
4     Boat or Ferry  0.07


---- Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.06
2          Restaurant  0.06
3  Italian Restaurant  0.04
4         Cheese Shop  0.04


---- CN Tower----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3             Plane  0.07
4     Boat or Ferry  0.07


---- Cabbagetown----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.07
2              Bakery  0.05
3  Italian Restaurant  0.05
4                 Pub  0.05


---- Central Bay Street----
                venue  freq
0         C

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = wt_grouped['Neighborhood']

for ind in np.arange(wt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(wt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Asian Restaurant,Clothing Store,Bakery,Bar
1,Bathurst Quay,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Sculpture Garden
2,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Pub,Café,Seafood Restaurant,Steakhouse,Beer Bar,Italian Restaurant,Farmers Market
3,CN Tower,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Sculpture Garden
4,Cabbagetown,Coffee Shop,Restaurant,Park,Café,Italian Restaurant,Pub,Pizza Place,Bakery,Gift Shop,Convenience Store


In [52]:
# set number of clusters
kclusters = 5

wt_grouped_clustering = wt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(wt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 4, 1, 4, 4, 3, 0, 3, 4], dtype=int32)

In [53]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

wt_merged = wt_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
wt_merged = wt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')

wt_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,4,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market,French Restaurant
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,4,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market,French Restaurant
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,3,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Pizza Place,Ramen Restaurant,Thai Restaurant,Theater,Tea Room
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,3,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Pizza Place,Ramen Restaurant,Thai Restaurant,Theater,Tea Room
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Coffee Shop,Restaurant,Café,Hotel,Bakery,Italian Restaurant,Park,Clothing Store,Gastropub,Pizza Place


In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(wt_merged['Latitude'], wt_merged['Longitude'], wt_merged['Neighborhood'], wt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [55]:
wt_merged.loc[wt_merged['Cluster Labels'] == 0, wt_merged.columns[[1] + list(range(5, wt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant,Athletics & Sports,Diner,Baby Store,Nightclub,Convenience Store


In [56]:
wt_merged.loc[wt_merged['Cluster Labels'] == 1, wt_merged.columns[[1] + list(range(5, wt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Downtown Toronto,1,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Sculpture Garden
24,Downtown Toronto,1,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Sculpture Garden
25,Downtown Toronto,1,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Sculpture Garden
26,Downtown Toronto,1,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Sculpture Garden
27,Downtown Toronto,1,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Sculpture Garden
28,Downtown Toronto,1,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Sculpture Garden
29,Downtown Toronto,1,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boutique,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Sculpture Garden


In [57]:
wt_merged.loc[wt_merged['Cluster Labels'] == 2, wt_merged.columns[[1] + list(range(5, wt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Downtown Toronto,2,Park,Playground,Trail,Women's Store,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [58]:
wt_merged.loc[wt_merged['Cluster Labels'] == 3, wt_merged.columns[[1] + list(range(5, wt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Pizza Place,Ramen Restaurant,Thai Restaurant,Theater,Tea Room
3,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Pizza Place,Ramen Restaurant,Thai Restaurant,Theater,Tea Room
8,Downtown Toronto,3,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Asian Restaurant,Clothing Store,Bakery,Bar
9,Downtown Toronto,3,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Asian Restaurant,Clothing Store,Bakery,Bar
10,Downtown Toronto,3,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Asian Restaurant,Clothing Store,Bakery,Bar
18,Downtown Toronto,3,Café,Restaurant,Bar,Japanese Restaurant,Coffee Shop,Bookstore,Bakery,Yoga Studio,Jazz Club,Beer Bar
19,Downtown Toronto,3,Café,Restaurant,Bar,Japanese Restaurant,Coffee Shop,Bookstore,Bakery,Yoga Studio,Jazz Club,Beer Bar
20,Downtown Toronto,3,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Chinese Restaurant,Burger Joint
21,Downtown Toronto,3,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Chinese Restaurant,Burger Joint
22,Downtown Toronto,3,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Chinese Restaurant,Burger Joint


In [59]:
wt_merged.loc[wt_merged['Cluster Labels'] == 4, wt_merged.columns[[1] + list(range(5, wt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market,French Restaurant
1,Downtown Toronto,4,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market,French Restaurant
4,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Hotel,Bakery,Italian Restaurant,Park,Clothing Store,Gastropub,Pizza Place
5,Downtown Toronto,4,Coffee Shop,Cocktail Bar,Restaurant,Pub,Café,Seafood Restaurant,Steakhouse,Beer Bar,Italian Restaurant,Farmers Market
6,Downtown Toronto,4,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Ice Cream Shop,Spa,Chinese Restaurant,Indian Restaurant,Middle Eastern Restaurant
11,Downtown Toronto,4,Coffee Shop,Aquarium,Hotel,Pizza Place,Café,Italian Restaurant,Scenic Lookout,Brewery,Restaurant,Bakery
12,Downtown Toronto,4,Coffee Shop,Aquarium,Hotel,Pizza Place,Café,Italian Restaurant,Scenic Lookout,Brewery,Restaurant,Bakery
13,Downtown Toronto,4,Coffee Shop,Aquarium,Hotel,Pizza Place,Café,Italian Restaurant,Scenic Lookout,Brewery,Restaurant,Bakery
14,Downtown Toronto,4,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Italian Restaurant,Gym,Gastropub,Deli / Bodega,Seafood Restaurant
15,Downtown Toronto,4,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Italian Restaurant,Gym,Gastropub,Deli / Bodega,Seafood Restaurant
